## Package Imports

In [3]:
#multiprocess inference: 
    #there is one analyze process running throughohut the duration of the program
        #main adds the microphone_data to a queue which analyze gets from the queue and analyzes
    #there is also one sms process running throughout the duration of the program
        #if a gunshot is detected w prob > 90%, a 1 is added to a queue which the sms alert process will
        #take as a sign to send a text
    #there is also one localization process running throughout the duration of the program
        #it receives audio data from the analysis process and currently writes a wav file containing
        #a gunshot and the 2s before and after any gunshot were predicted
        #uses threshold values to determine where in the clip the gunshots took place

import pyaudio
import librosa
import logging
import multiprocessing
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from multiprocessing import Process, active_children
from tensorflow.keras import Input, layers, optimizers, backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import IPython.display as ipd
import matplotlib.pyplot as plt
#from gsmmodem.modem import GsmModem
import time
import soundfile as sf
from array import array
from sys import byteorder


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/display.py:32: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  mpl.rcParams.update(**_matplotlibrc)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:849: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  self[key] = value
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:849: MatplotlibDeprecationWarning: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  self[key] = value
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:849: MatplotlibDeprecationWarning: 
The 

## Set up Logging

In [4]:
#stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger('debugger')
logger.setLevel(logging.DEBUG)
ch = logging.FileHandler('spam.log')
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

## Variable Initializations

In [5]:
#on the pi, change to paInt16
audio_format = pyaudio.paInt16
audio_rate = 44100
audio_channels = 1
audio_device_index = 0
audio_frames_per_buffer = 4410
audio_sample_duration = 2
input_shape = (audio_rate, 1)
phone_numbers_to_message = ["8163449956", "9176202840", "7857642331"]

## Model Initializer Functions

In [22]:
def auc(y_true, y_pred):
        auc = tf.metrics.auc(y_true, y_pred)[1]
        K.get_session().run(tf.local_variables_initializer())
        return auc

In [23]:
def load_model_one(weights_file):
    # 1D Time-Series Model Parameters
    drop_out_rate = 0.1
    learning_rate = 0.001
    number_of_classes = 2
    optimizer = optimizers.Adam(learning_rate, learning_rate / 100)
    input_shape = (44100, 1)
    input_tensor = Input(shape=input_shape)
    metrics = [auc, "accuracy"]
    
    x = layers.Conv1D(16, 9, activation="relu", padding="same")(input_tensor)
    x = layers.Conv1D(16, 9, activation="relu", padding="same")(x)
    x = layers.MaxPool1D(16)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool1D(4)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool1D(4)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Dropout(rate=(drop_out_rate * 2))(x) # Increasing drop-out rate here to prevent overfitting

    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(1028, activation="relu")(x)

    output_tensor = layers.Dense(number_of_classes, activation="softmax")(x)

    model = tf.keras.Model(input_tensor, output_tensor)
    
    model.compile(optimizer=optimizer, loss=keras.losses.binary_crossentropy, metrics=metrics)

    model.load_weights(weights_file)
    
    return model

In [24]:
def load_model_two(weights_file):
    # Loading 1D Time-Series Model Weights
   
    #2D Spectrogram Model Parameters
    input_shape = (128, 87, 1)
    input_tensor = Input(shape=input_shape)
    learning_rate = 0.001
    optimizer = optimizers.Adam(learning_rate, learning_rate / 100)
    filter_size = (3,3)
    maxpool_size = (3,3)
    activation = "relu"
    drop_out_rate = 0.1
    number_of_classes = 2
    metrics = [auc, "accuracy"]

    # Loading 2D Spectrogram Model
    x = layers.Conv2D(16, filter_size, activation=activation, padding="same")(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(maxpool_size)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv2D(32, filter_size, activation=activation, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(maxpool_size)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv2D(64, filter_size, activation=activation, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(maxpool_size)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv2D(256, filter_size, activation=activation, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalMaxPool2D()(x)
    x = layers.Dropout(rate=(drop_out_rate * 2))(x) # Increasing drop-out rate here to prevent overfitting

    x = layers.Dense(64, activation=activation)(x)
    x = layers.Dense(1028, activation=activation)(x)
    output_tensor = layers.Dense(number_of_classes, activation="softmax")(x)

    spec_model = tf.keras.Model(input_tensor, output_tensor)
    spec_model.compile(optimizer=optimizer, loss=keras.losses.binary_crossentropy, metrics=metrics)

    # Loading 2D Spectrogram Model Weights
    spec_model.load_weights(weights_file)
    return spec_model

## Process 1: Analuze

In [25]:
#loads model
#if microphone data reaches a threshold, predicts with the model
#if model predicts a gunshot, send a text, and add data to the time localization queue

def analyze_microphone_data(audio_rate, ):
    
    #LOAD THE MODEL
    model = load_model_one("./models/gunshot_sound_model.h5")
    #model = load_model_two("./models/gunshot_sound_model_spectrograph_model.h5")
    
    #booleans to keep track of detection state
    detected = False
    detected_first = False
    detected_again = False
    end_of_detections = False
    
    #initialize prev mic data as an arrat of just zeros
    prev_mic_data = np.zeros((88200,), dtype = np.int16)
    
    #infinite loop
    while True:
        #will wait until something is in the queue to continue
        microphone_data = audio_analysis_queue.get()
        time_of_sample = audio_analysis_queue.get()
        logger.debug("time: " + str(time_of_sample))
        
        #if reaches a threshold, analyze and update booleans
        if max(microphone_data) >= 1000:
            # Performs post-processing on live audio samples
            reformed_microphone_data = librosa.resample(y=microphone_data, orig_sr=audio_rate, target_sr=22050)
            reformed_microphone_data = librosa.util.normalize(reformed_microphone_data)
            reformed_microphone_data = reformed_microphone_data[:audio_rate]
            reformed_microphone_data = reformed_microphone_data.reshape(-1, audio_rate, 1)

            # Passes a given audio sample into the model for prediction
            probabilities = model.predict(reformed_microphone_data)
            logger.debug("Probabilities derived by the model: " + str(probabilities))

            #send a text if gunshot if detected
            #update booleans to show state of detection
            if (probabilities[0][1] >= 0.9):
                logger.debug("   Probability of gunshot > 0.9")
                sms_alert_queue.put("1")
                if  not detected:
                    detected = True
                    detected_first = True
                else:
                    detected_first = False
                    detected_again = True
            else:
                if detected:
                    end_of_detections = True
                else:
                    end_of_detections = False
                detected = False
                detected_first = False
                detected_again = False
        
        #otherwise, make sure you update the booleans still
        else:
            if detected:
                end_of_detections = True
            else:
                end_of_detections = False
            detected = False
            detected_first = False
            detected_again = False
            
        #based on booleans, put microphone data or prev mic data on the time localization queue
        if detected_first:
            localization_data_queue.put(prev_mic_data)
            localization_data_queue.put(microphone_data)
            localization_time_queue.put(time_of_sample)
        if detected_again:
            localization_data_queue.put(microphone_data)
        if end_of_detections:
            localization_data_queue.put(microphone_data)
            localization_data_queue.put(np.zeros((44100,), dtype = np.int16))
            end_of_detections = False

                    
        #keep track of the previous microphone data
        prev_mic_data = microphone_data
        


## Process 3: Figure out exact time of gunshot

In [26]:
#get data from the time localization queue
#writes out a wav file including all gunshot audio + 2seconds before and after

def localize():
    
    #variables
    all_mic_data = []
    #index of iteration
    loop = 0
    
    #infinite loop
    while True:
        #get all the mic data from the queue and add it to the list
        while True:
            chunk = localization_data_queue.get()
            if np.array_equal(chunk, (np.zeros((44100,), dtype = np.int16))):
                logger.debug("    reached end of a gunshot group")
                break;
            else:
                all_mic_data.append(chunk) 

                
        #get beginning of mic data times
        time_at_beg_of_first_gunshot_clip = localization_time_queue.get()
        
        # processing
        np_all_mic_data = np.concatenate(all_mic_data)
        ref_all_mic_data = librosa.resample(y=np_all_mic_data, orig_sr=44100, target_sr=22050)
        ref_all_mic_data = librosa.util.normalize(ref_all_mic_data)
        
        #write out a soundfile of the entire chunk
        with sf.SoundFile("/Users/laurenogden/Downloads/" + str(loop) + + "_" + time.ctime(time_at_beg_of_first_gunshot_clip)".wav", mode='wb', samplerate=22050, channels=1) as file:
            file.write(ref_all_mic_data)
        logger.debug("    wrote out the #" + str(loop) + "chunk, size: " + str(len(ref_all_mic_data)))

 
        #figure out the times of the gunshots in the clip
        #sort the data, figure out the threshold
        sorted_data = np.sort(ref_all_mic_data)
        threshold = sorted_data[len(sorted_data) - int(len(sorted_data)*0.001)]

        #find all values above that threshold
        above_threshold = []
        for i in range(0, len(ref_all_mic_data)):
            if ref_all_mic_data[i] > threshold:
                above_threshold.append(i)
                
        #separate out individual gunshots from that whole chunk
        distinct_shots = []
        distinct_shots.append(above_threshold[0])
        for i in range(1, len(above_threshold)):
            #if within 5ms of each other, assume from same shot
            if above_threshold[i] - above_threshold[i-1] > 0.05*22050:
                distinct_shots.append(above_threshold[i])

        #times relative to beginning of the saved clip
        logger.debug("There were " + str(len(distinct_shots)) + " distinct shots detected at " + str(time.ctime(time_at_beg_of_first_gunshot_clip)))
        for i in distinct_shots:
            logger.debug(i/22050)
            
        #times in general
        #logger.debug("The exact times of the gunshots were: ")
        #for i in distinct_shots:
            #logger.debug(time_at_beg_of_first_gunshot_clip + i/22050 - 2)
            #logger.debug(time.ctime(time_at_beg_of_first_gunshot_clip + i/22050 - 2))

            
        #clear all mic data to be ready for the next clip
        del all_mic_data[:]
        #increase index
        loop += 1
        
        



## Process 2: Send SMS

In [27]:
# send an sms if a gunshot is detected

def send_sms_alert(phone_numbers_to_message, ):
    
    # Modem variables
    modem_port = '/dev/ttyUSB0'
    modem_baudrate = 115200
    modem_sim_pin = None # SIM card PIN (if any)
    
    # Connect to the modem
    #print("Initializing connection to modem...")
    #modem = GsmModem(modem_port, modem_baudrate)
    #modem.smsTextMode = False
    #modem.connect(modem_sim_pin)
    
    
    # If the model detects a gunshot, an SMS alert will be sent to local authorities
    while True:
        sms_alert_status = sms_alert_queue.get()
        if sms_alert_status == "1":
            logger.debug("~~~~~~~~pretend like I just sent a text message~~~~~~~~~~")
            '''
            try:
                modem.waitForNetworkCoverage(timeout=86400)
                message = " (Testing) ALERT: A Gunshot Has Been Detected (Testing)"
                for number in phone_numbers_to_message:
                    modem.sendSms(number, message)
                logger.debug(" *** Sent out an SMS alert to all designated recipients *** ")
            except:
                logger.debug("ERROR: Unable to successfully send an SMS alert to the designated recipients.")
                pass
            finally:
                logger.debug(" * Finished evaluating an audio sample with the model * ")
            '''

## Main Process: Capture Audio

In [28]:
#open the stream
pa = pyaudio.PyAudio()
    
stream = pa.open(format = audio_format,
                 rate = audio_rate,
                 channels = audio_channels,
                 input_device_index = audio_device_index,
                 frames_per_buffer = audio_frames_per_buffer,
                 input = True)

In [29]:
if __name__ == '__main__':
    
    #Create the Queues
    sms_alert_queue = multiprocessing.Queue()
    audio_analysis_queue = multiprocessing.Queue()
    localization_data_queue = multiprocessing.Queue()
    localization_time_queue = multiprocessing.Queue()
    
    #Create Analysis Process
    analysis_process = Process(target = analyze_microphone_data, args = (audio_rate, ))
    analysis_process.start()
    
    #Create SMS Process
    sms_alert_process = Process(target = send_sms_alert, args = (phone_numbers_to_message, ))
    sms_alert_process.start()
    
    #Create process for localization
    localization_process = Process(target = localize, args = ())
    localization_process.start()
    

    logger.debug("--- Recording Audio ---")
    
    while(True):
        sound_data = array('h')
        
        now = time.time()

        # Loops through the stream and appends audio chunks to the frame array
        for i in range(0, int(audio_rate / audio_frames_per_buffer * audio_sample_duration)):
            sound_buffer = array('h', stream.read(audio_frames_per_buffer, exception_on_overflow = False))
            if byteorder == 'big':
                sound_buffer.byteswap()
            sound_data.extend(sound_buffer)
    
        microphone_data = np.array(sound_data)
        
        #emit_string = "Cumulative length of a given two-second audio sample: " + str(len(microphone_data))
        #logger.debug(emit_string)
        #emit_string = "The maximum frequency value for a given two-second audio sample: " + str(max(microphone_data))
        #logger.debug(emit_string)
        
        
        # put microphone data on the queue to go to the analysis process
            # the analysis process will check the threshold: if max(microphone_data) >= 1000:
        audio_analysis_queue.put(microphone_data)
        #also put time: for localization
        audio_analysis_queue.put(now)

        
        # Closes all finished processes    
        #kids = active_children()
        #logger.debug("these are my active children: ")
        #logger.debug(kids)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Process Process-4:
Process Process-3:
Process Process-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/proc

KeyboardInterrupt: 